# Calibrating the Radar Rotation Angles

In [ ]:
%load_ext autoreload
%autoreload 2

# find the root of the project
import os
from pathlib import Path

ROOT = Path(os.getcwd()).parent
while not ROOT.joinpath(".git").exists():
    ROOT = ROOT.parent

# add the root to the python path
import sys

sys.path.append(str(ROOT))

import dotenv

dotenv.load_dotenv(ROOT.joinpath(".env"))

## Pull the Calibrate Lane Centers from Felt

In [ ]:
import requests as re
import geopandas as gpd

# get teh

# from https://feltmaps.notion.site/Getting-Started-With-The-Felt-API-69c8b02b7d8e436daa657a04a2dbaffa#cf165f809cbf4e2f9a4bba3bf00e29c1
url = "https://felt.com/api/v1/maps/DRIVER-Lane-Centers-888IuX77T6KfCbLlEraVTD/elements"
bearer = "Bearer " + os.getenv("FELT_API_TOKEN")
headers = {"Content-Type": "application/json", "authorization": bearer}


vals = re.get(url, headers=headers)

In [ ]:
from io import StringIO
import pandas as pd


lane_geometry = gpd.read_file(StringIO(vals.text)).query("calibrate == True")

len_before = len(lane_geometry)

# split the multi-linestring into individual linestrings
lane_geometry = lane_geometry.explode(index_parts=False).reset_index(drop=True)

len_after = len(lane_geometry)

assert len_before == len_after

lane_geometry.explore()

## Interpolate the Lane Centers

In [ ]:
from shapely.geometry import LineString


# interpolate the LineStrings to 1m
# https://gis.stackexchange.com/a/367965
def redistribute_vertices(geom, distance):
    if geom.geom_type == "LineString":
        num_vert = int(round(geom.length / distance))
        if num_vert == 0:
            num_vert = 1
        return LineString(
            [
                geom.interpolate(float(n) / num_vert, normalized=True)
                for n in range(num_vert + 1)
            ]
        )
    elif geom.geom_type == "MultiLineString":
        parts = [redistribute_vertices(part, distance) for part in geom]
        return type(geom)([p for p in parts if not p.is_empty])
    else:
        raise ValueError("unhandled geometry %s", (geom.geom_type,))


old_crs = lane_geometry.crs
lane_geometry = lane_geometry.to_crs(lane_geometry.estimate_utm_crs())
lane_geometry = lane_geometry.set_geometry(
    lane_geometry.geometry.apply(lambda x: redistribute_vertices(x, 1))
)
lane_geometry = lane_geometry.to_crs(old_crs)


# lane_geometry = lane_geometry.loc[~lane_geometry.index.isin([2, 3,])]

In [ ]:
lane_geometry.explore()

In [ ]:
# fit b-spline to all the lanes & interpolate every 0.1m
from src.frenet import SplineLane
import numpy as np

utm_crs = lane_geometry.estimate_utm_crs()

splines = [
    SplineLane(
        name=f"lane - {ind}",
        centerline=np.stack(lane["geometry"].coords.xy).T,
        crs=utm_crs,
    )
    .fit(
        k=3,
        s=None,
    )
    .interpolate(0.1)
    for ind, lane in lane_geometry.to_crs(utm_crs).iterrows()
]

In [ ]:
import plotly.express as px

interpolated_lane_df = pd.concat(
    [spline.to_gdf(linestring=True).to_crs(lane_geometry.crs) for spline in splines],
)

interpolated_lane_df.explore(
    max_zoom=20,
)

## Use the Lanes to Calibrate the Radar Data

In [ ]:
from src.radar import Filtering


f = Filtering(
    ROOT / "geo_data" / "surveyed_origins.json",
    ROOT / "geo_data" / "network_outline.geojson",
)

f_calibrated = Filtering(
    ROOT / "geo_data" / "calibrated_origins.json",
    ROOT / "geo_data" / "network_outline.geojson",
)

### Load Radar Data

In [ ]:
import polars as pl

In [ ]:
RADAR_DIR = Path("/DOECV2X/Radar") / "all_working"
print(RADAR_DIR)

# map to a file for speed
tmp_file = ROOT / "tmp" / f"{RADAR_DIR.stem}.parquet"
tmp_file.parent.mkdir(exist_ok=True, parents=True)

radar_df = pl.scan_parquet(tmp_file)

In [ ]:
radar_df = (
    radar_df.pipe(f.create_object_id)
    .pipe(f.filter_network_boundaries)
    .sort(by=["object_id", "epoch_time"])
    .set_sorted(["object_id", "epoch_time"])
    .collect()
)

In [ ]:
radar_df.shape

### Calibrate the Radar Data

In [ ]:
line_point_df = pd.concat(
    [spline.to_gdf().to_crs(lane_geometry.crs) for spline in splines],
)

In [ ]:
line_point_df

In [ ]:
from shapely.geometry import Point
from scipy.spatial import KDTree

### Radar 1

In [ ]:
from typing import List


def rotate_array(arr: np.ndarray, angle: float) -> np.ndarray:
    """Rotate a 2d array by a given angle in radians"""
    rot_matrix = np.array(
        [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]
    ).squeeze()
    return (rot_matrix @ arr.T).T


def move_origin(
    arr: np.ndarray, x_correction: float, y_correction: float, origins: np.ndarray
) -> np.ndarray:
    """Move the origin of a 2d array"""
    return arr + (np.array([x_correction, y_correction]) + origins)


def calculate_loss(
    sol, arr: np.ndarray, utm_origin: np.ndarray, tree: KDTree, scores: List = None, angle: float = None
) -> float:
    """Calculate the loss for a given angle"""
    if scores is None:
        scores = []
    # angle, x_corr, y_corr = sol
    x_corr, y_corr = sol
    rotated_arr = rotate_array(arr, angle)
    rotated_arr = move_origin(rotated_arr, x_corr, y_corr, utm_origin)

    distances = np.zeros(len(rotated_arr)) + np.inf

    # for lane in splines:

    #     _, d = lane.snap_points(
    #         rotated_arr,
    #         return_all=False,
    #         max_dist=2,
    #     )

    #     distances = np.where(np.abs(d) < distances, d, distances)

    #     # distances = np.min(distances, np.abs(d)) * np.where(d < 0, -1, 1)

    distances = tree.query(rotated_arr, workers=-1, distance_upper_bound=8)[0]

    distances = distances[distances != np.inf]  # remove inf values
    mean_distance_penalty = np.quantile(distances, 0.10)
    # add matched point penalty
    matched_points = (len(arr) - len(distances)) / len(arr)
    # score = matched_points + mean_distance_penalty
    score = mean_distance_penalty
    # score = 
    scores.append((score, angle, x_corr, y_corr))
    return score

In [ ]:
from scipy.optimize import minimize


radar_df = (
    radar_df.with_columns((1 / pl.count()).over("h3").alias("weight"))
    .with_columns(
        (
            (pl.col("f32_positionX_m") ** 2 + pl.col("f32_positionY_m") ** 2) ** 0.5
        ).alias("distance")
    )
    .filter(pl.col("distance").is_between(50, 300))
    .filter(pl.col('f32_velocityInDir_mps') > 2)
)


overriden_start_points = {"10.160.7.141": (0, 0), "10.160.7.146": (0, 0)}


def optimize(radar: str):
    tl_df = (
        radar_df.filter(pl.col("ip").str.contains(radar))
        # .sample(
        #     1_000_000
        # )
        # .to_pandas()
    )
    # tl_df = tl_df.sample(
    #     frac=0.9,
    #     weights="weight",
    # )

    data_array = tl_df[["f32_positionX_m", "f32_positionY_m"]].to_numpy()

    # create a circle with radius 400m around the origin
    utm_c = f_calibrated.radar_locations[radar]

    origin = Point(utm_c[0], utm_c[1])
    origin = origin.buffer(300)

    tl_geometry = line_point_df.to_crs(utm_crs).clip(origin).drop(columns=["geometry"])
    true_x = tl_geometry["x"]  # - utm_c[0]
    true_y = tl_geometry["y"]  # - utm_c[1]

    # build a KDTree for the lane points
    tree = KDTree(np.stack([true_x, true_y]).T)
    # tree = None

    scores = []

    if radar in overriden_start_points:
        x, y = overriden_start_points[radar]
    else:
        x, y = 0, 0

    res = minimize(
        calculate_loss,
        # x0=[f.rotations[radar], x, y],
        x0=[x, y],
        # x0=[f.rotations[radar]],
        args=(data_array, np.array([utm_c[0], utm_c[1]]), tree, scores, f_calibrated.rotations[radar]),
        bounds=[
            # (f_calibrated.rotations[radar] - (np.pi / 4), f_calibrated.rotations[radar] + (np.pi / 4)),
            (-4, 4),
            (-4, 4),
        ],
        tol=1e-9
        # method="Nelder-Mead",
    )

    return {
        # "angle": res.x[0],
        "x_correction": res.x[0],
        "y_correction": res.x[1],
        "scores": scores,
    }


results = {radar: optimize(radar) for radar in f.radar_locations.keys()}
# results = {radar: optimize(radar) for radar in ['10.160.7.146']}

#### Rotated v. Original Radar Data

In [ ]:
for radar, res in results.items():
    print(f"{radar}: {res['x_correction']}, {res['y_correction']}")

In [ ]:
plot_radar = "10.160.7.141"

plot_df = (
    radar_df.filter(pl.col("ip").str.contains(plot_radar))
    # .to_pandas()
    .sample(10_000, )
)

optimal_df = plot_df.pipe(
    f.rotate_radars,
    # rotations={plot_radar: results[plot_radar]["angle"]},
).pipe(
    f.update_origin,
    locations={
        plot_radar: (
            f_calibrated.radar_locations[plot_radar][0] + results[plot_radar]["x_correction"],
            f_calibrated.radar_locations[plot_radar][1] + results[plot_radar]["y_correction"],
        )
    },
)

before_df = plot_df.pipe(
    f_calibrated.rotate_radars,
).pipe(
    f_calibrated.update_origin,
)


optimal_df = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(
        optimal_df["utm_x"],
        optimal_df["utm_y"],
    ),
    crs=utm_crs,
)

before_df = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(
        before_df["utm_x"],
        before_df["utm_y"],
    ),
    crs=utm_crs,
)


m = lane_geometry.explore(
    max_zoom=26,
)

optimal_df.explore(
    m=m,
    color="red",
)

before_df.explore(
    m=m,
    color="purple",
)

## Save the Calibrated Radar Data

In [ ]:
import utm

results_origin = {
    radar: {
        "origin": utm.to_latlon(
            f.radar_locations[radar][0] + results[radar]["x_correction"],
            f.radar_locations[radar][1] + results[radar]["y_correction"],
            *f.radar_locations[radar][2:]
        )[::-1],
        "angle": np.rad2deg(results[radar]["angle"]),
    }
    for radar, _ in results.items()
}

In [ ]:
import json

with open(ROOT / "geo_data" / "calibrated_origins_10_31.json", "w") as f_:
    json.dump(results_origin, f_, indent=4)

### Save Some of the Data for Testing

In [ ]:
f_calibrated = Filtering(
    ROOT / "geo_data" / "calibrated_origins_10_31.json",
    ROOT / "geo_data" / "network_outline.geojson",
)

In [ ]:
radar_df.sample(100_000).pipe(
    f_calibrated.rotate_radars,
).pipe(
    f_calibrated.update_origin,
).pipe(f.radar_to_latlon).select(
    [
        "ip",
        "lat",
        "lon",
    ]
).write_csv(
    "tmp.csv"
)